In [1]:
%matplotlib inline

In [2]:
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
#from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal, ndimage, spatial
from scipy.signal import correlate
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter
import math 

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, auc

from scipy.interpolate import interp1d
import scipy.stats as stats
from scipy.stats import spearmanr
import seaborn as sns

In [3]:
file = open(r"/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/facial_tracking_data_all.txt","r") 

print(file.read())

./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_050917_OME notes.pdf
./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_050917_OME.pdf
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_NORMAL.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_KA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_PA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_PATAKA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_TA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_BLOW.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_KISS.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_OPEN.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_SPREAD.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_RST_REST.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_LATFAST.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_LATNORMAL.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_NOSE.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_OUT.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_PROTRUSION.rssdk
./ALS/AF01_SBK_20170509/02/RS

In [4]:
data = pd.read_csv("/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/facial_tracking_data_all.txt",  header=None)

In [5]:
print(data.shape)
data

(5804, 1)


,0
0,./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_05091...
1,./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_05091...
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_K...
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...
...,...
5799,./DATASET_updated_20191018.xlsx
5800,./Acoustic_DataAcquisitionLog.xlsx
5801,./content_testing.txt
5802,./._content_testing.txt


In [6]:
tasks = ["BBP_NORMAL" ,"DDK_PA" , "NSM_BIGSMILE" ,"NSM_OPEN" , "NSM_SPREAD" , "RST_REST" ,"NSM_KISS"]




def wanted_files(row):
       
    if '.rssdk' in os.path.basename(row) or '.bag' in os.path.basename(row):
        return os.path.basename(row)
    else:
        return None
        
def get_subject_id(row):
    
    return row.split('_')[0]        
        

def set_task(row):
    for t in tasks:
        if t in row:
            return t
  

data.rename(columns = {0: 'file_path'}, inplace = True)


print(data.shape)
data['file_name'] = [wanted_files(x) for x in data['file_path']]

print(data.shape)
data = data[data['file_name'].notna()]
print(data.shape)
data['subject_id'] = [get_subject_id(x) for x in data['file_name']]
data['task'] = [set_task(x) for x in data['file_name']]
data = data[data['task'].notna()]


print(data.shape)
print(data.task.unique())
data

(5804, 1)
(5804, 2)
(4357, 2)
(1135, 4)
['BBP_NORMAL' 'DDK_PA' 'NSM_KISS' 'NSM_OPEN' 'NSM_SPREAD' 'RST_REST'
 'NSM_BIGSMILE']


<ipython-input-6-159ee699571d>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['subject_id'] = [get_subject_id(x) for x in data['file_name']]
<ipython-input-6-159ee699571d>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['task'] = [set_task(x) for x in data['file_name']]


,file_path,file_name,subject_id,task
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA
5,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA
8,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS
9,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN
...,...,...,...,...
5776,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST
5795,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS
5796,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN
5797,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD


In [7]:
# data that already has clinical score 

path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/ALS/SLP_Assessment_ALS.csv'
df_als = pd.read_csv(path)

df_als = df_als.drop('Unnamed: 8', axis=1)

#print(df_als.shape)
#print(df_als.columns)
#print(df_als['Subject ID'].unique())
#df_als.head()

# STROKE DATA
path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/Stroke/SLP_Assessment_PS.csv'
df_ps = pd.read_csv(path)

df_ps = df_ps.drop('Unnamed: 8', axis=1)

#print(df_ps.shape)
#print(df_ps.columns)
#print(df_ps['Subject ID'].unique())
#df_ps.head()

# HEALTHY CONTROL DATA
path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/Healthy controls/SLP_Assessment_HC.csv'
df_hc = pd.read_csv(path)

df_hc = df_hc.drop('Unnamed: 8', axis=1)
df_hc = df_hc.drop('Unnamed: 15', axis=1)
#print(df_hc.shape)
#print(df_hc.columns)
#print(df_hc['Subject ID'].unique())




slp_scores = pd.concat([df_als, df_ps, df_hc], ignore_index=True)
print(slp_scores.shape)
#slp_scores['File Name'] = [wanted_tasks_slp(x) for x in slp_scores['File Name']]

slp_scores['task'] = [set_task(x) for x in slp_scores['File Name']]

slp_scores = slp_scores[slp_scores['task'].notna()]

print(slp_scores.shape)

slp_scores

(261, 14)
(222, 15)


,File Name,Subject ID,Symmetry (SLP1),ROM (SLP1),Speed (SLP1),Variability (SLP1),Fatigue (SLP1),Tot (SLP1),Symmetry (SLP2),ROM (SLP2),Speed (SLP2),Variability (SLP2),Fatigue (SLP2),Tot (SLP2),task
0,A002_02_BBP_NORMAL_color.avi,A002,1,1,1,1,1,5,2,1,1,1,1,6,BBP_NORMAL
1,A002_02_DDK_PATAKA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9,DDK_PA
2,A002_02_DDK_PA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9,DDK_PA
4,A002_02_NSM_KISS_color.avi,A002,2,1,1,1,4,9,1,1,1,1,3,7,NSM_KISS
5,A002_02_NSM_OPEN_color.avi,A002,1,1,1,1,1,5,1,1,1,1,1,5,NSM_OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,N019_02_DDK_PA_color.avi,N019,2,1,1,1,1,6,1,1,1,1,1,5,DDK_PA
256,N019_02_NSM_BIGSMILE_color.avi,N019,1,2,2,1,1,7,2,2,2,2,1,9,NSM_BIGSMILE
258,N019_02_NSM_KISS_color.avi,N019,2,1,2,2,1,8,2,1,1,1,1,6,NSM_KISS
259,N019_02_NSM_OPEN_color.avi,N019,2,1,1,1,1,6,2,1,1,1,1,6,NSM_OPEN


In [8]:
print(slp_scores['Subject ID'].unique())
print(len(slp_scores['Subject ID'].unique()))

def id_label_change(sub_id):
    s = sub_id
    if 'N' in sub_id:         
        s = s.replace('N0', 'NF')
    elif 'A' in sub_id:
        s = s.replace('A0', 'AF')

    return s

slp_scores['Subject ID'] = [id_label_change(x) for x in slp_scores['Subject ID']]
slp_scores['task'] = [set_task(x) for x in slp_scores['File Name']]



print(len(slp_scores['Subject ID'].unique()))
slp_scores['Subject ID'].unique()

['A002' 'A006' 'A008' 'A009' 'A010' 'A011' 'A012' 'A014' 'A015' 'A016'
 'A017' 'OP01' 'OP02' 'OP03' 'S001' 'S002' 'S003' 'S005' 'S006' 'S007'
 'S008' 'S009' 'S011' 'S012' 'S013' 'N001' 'N002' 'N003' 'N004' 'N007'
 'N008' 'N010' 'N011' 'N012' 'N017' 'N019']
36
36


array(['AF02', 'AF06', 'AF08', 'AF09', 'AF10', 'AF11', 'AF12', 'AF14',
       'AF15', 'AF16', 'AF17', 'OP01', 'OP02', 'OP03', 'S001', 'S002',
       'S003', 'S005', 'S006', 'S007', 'S008', 'S009', 'S011', 'S012',
       'S013', 'NF01', 'NF02', 'NF03', 'NF04', 'NF07', 'NF08', 'NF10',
       'NF11', 'NF12', 'NF17', 'NF19'], dtype=object)

In [9]:
print(slp_scores.shape)
print(data.shape)
print(len(data['subject_id'].unique()))
data['subject_id'].unique()

(222, 15)
(1135, 4)
88


array(['AF01', 'AF02', 'AF03', 'AF04', 'AF05', 'AF06', 'AF07', 'AF09',
       'AF10', 'AF11', 'AF12', 'AF13', 'AF14', 'AF16', 'AF17', 'AF18',
       'AF19', 'ASH1', 'NF00', 'NF01', 'NF02', 'NF03', 'NF04', 'NF05',
       'NF06', 'NF07', 'NF08', 'NF09', 'NF10', 'NF11', 'NF12', 'NF13',
       'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21',
       'NF22', 'NF23', 'NF24', 'NF25', 'NF26', 'NF27', '001', '002',
       'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06',
       'PD07', 'PD08', 'P002', 'P003', 'P004', 'OP01', 'OP02', 'OP03',
       'OP04', 'OP05', 'OP06', 'OP07', 'OP08', 'OP09', 'OP10', 'OP11',
       'OP12', 'OP13', 'OP14', 'S001', 'S002', 'S003', 'S005', 'S006',
       'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S004'],
      dtype=object)

In [10]:
test = data.merge(slp_scores, how= 'left', left_on=['subject_id', 'task'], right_on=['Subject ID', 'task'])
print(test.shape)
test.head()

(1245, 18)


,file_path,file_name,subject_id,task,File Name,Subject ID,Symmetry (SLP1),ROM (SLP1),Speed (SLP1),Variability (SLP1),Fatigue (SLP1),Tot (SLP1),Symmetry (SLP2),ROM (SLP2),Speed (SLP2),Variability (SLP2),Fatigue (SLP2),Tot (SLP2)
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test.columns

Index(['file_path', 'file_name', 'subject_id', 'task', 'File Name',
       'Subject ID', 'Symmetry (SLP1)', 'ROM (SLP1)', 'Speed (SLP1)',
       'Variability (SLP1)', 'Fatigue (SLP1)', 'Tot (SLP1)', 'Symmetry (SLP2)',
       'ROM (SLP2)', 'Speed (SLP2)', 'Variability (SLP2)', 'Fatigue (SLP2)',
       'Tot (SLP2)'],
      dtype='object')

In [12]:
len(set(data['subject_id']).intersection(set(slp_scores['Subject ID'])))
#set(data['subject_id']).intersection(set(slp_scores['Subject ID']))

34

In [13]:
test['Need_Clinical_Score'] = np.where(test['Tot (SLP1)'].isnull() , 1, 0)
#test['Need_Clinical_Score'].loc[test['Tot (SLP1)'] == None] = 1
#test['Need_Clinical_Score'].loc[test['Need_Clinical_Score'] != 1] = 0

test = test.drop(['File Name',
       'Subject ID', 'Symmetry (SLP1)', 'ROM (SLP1)', 'Speed (SLP1)',
       'Variability (SLP1)', 'Fatigue (SLP1)', 'Tot (SLP1)', 'Symmetry (SLP2)',
       'ROM (SLP2)', 'Speed (SLP2)', 'Variability (SLP2)', 'Fatigue (SLP2)',
       'Tot (SLP2)'], axis = 1)

test

,file_path,file_name,subject_id,task,Need_Clinical_Score
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,1
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,1
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,1
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,1
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,1
...,...,...,...,...,...
1240,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST,1
1241,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS,1
1242,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN,1
1243,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD,1


In [14]:
test.subject_id.unique()

array(['AF01', 'AF02', 'AF03', 'AF04', 'AF05', 'AF06', 'AF07', 'AF09',
       'AF10', 'AF11', 'AF12', 'AF13', 'AF14', 'AF16', 'AF17', 'AF18',
       'AF19', 'ASH1', 'NF00', 'NF01', 'NF02', 'NF03', 'NF04', 'NF05',
       'NF06', 'NF07', 'NF08', 'NF09', 'NF10', 'NF11', 'NF12', 'NF13',
       'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21',
       'NF22', 'NF23', 'NF24', 'NF25', 'NF26', 'NF27', '001', '002',
       'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06',
       'PD07', 'PD08', 'P002', 'P003', 'P004', 'OP01', 'OP02', 'OP03',
       'OP04', 'OP05', 'OP06', 'OP07', 'OP08', 'OP09', 'OP10', 'OP11',
       'OP12', 'OP13', 'OP14', 'S001', 'S002', 'S003', 'S005', 'S006',
       'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S004'],
      dtype=object)

### Need Parsing ?

stategy: from the hard drive: go through Diego's data and write all the .Table files in a csv.
then read the csv here, extract task and subject id from the csv 
BE CAREFUL about the OP data as the date matters 
then merge the diego to test 
write test into csv 


In [15]:
file = open(r"/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/parsed_data_diego.txt","r") 

print(file.read())

./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table
./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table
./ALS/new/AF13/A013_02_NSM_OPEN_video.Table
./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table
./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table
./ALS/new/AF14/A014_02_NSM_BIGSMILE_video.Table
./ALS/new/AF14/A014_02_NSM_OPEN_video.Table
./ALS/new/AF14/A014_02_NSM_SPREAD_video.Table
./ALS/new/AF16/A016_02_BBP_NORMAL_video.Table
./ALS/new/AF16/A016_02_NSM_BIGSMILE_video.Table
./ALS/new/AF16/A016_02_NSM_OPEN_video.Table
./ALS/new/AF16/A016_02_NSM_SPREAD_video.Table
./ALS/new/AF16/A016_02_RST_REST3_video.Table
./ALS/new/AF17/A017_02_BBP_NORMAL_video.Table
./ALS/new/AF17/A017_02_NSM_BIGSMILE_video.Table
./ALS/new/AF17/A017_02_NSM_OPEN_video.Table
./ALS/new/AF17/A017_02_NSM_SPREAD_video.Table
./ALS/new/AF17/A017_02_RST_REST3_video.Table
./ALS/new/AF18/AF18_02_NSM_BIGSMILE_video.Table
./ALS/new/AF18/AF18_02_NSM_OPEN_video.Table
./ALS/new/AF18/AF18_02_NSM_SPREAD_video.Table
./ALS/new/AF18/AF18_02_RST_REST_vide

In [19]:
parsed = pd.read_csv("/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/parsed_data_diego.txt",  header=None)
print(parsed.shape)
parsed

(236, 1)


,0
0,./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table
1,./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table
2,./ALS/new/AF13/A013_02_NSM_OPEN_video.Table
3,./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table
4,./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table
...,...
231,./Post-Stroke/old/S012/S012_02_NSM_OPEN_video....
232,./Post-Stroke/old/S012/S012_02_NSM_SPREAD_vide...
233,./Post-Stroke/old/S013/S013_02_BBP_NORMAL_audi...
234,./Post-Stroke/old/S013/S013_02_NSM_OPEN_video....


In [23]:
parsed.rename(columns = {0: 'file_path'}, inplace = True)

parsed['file_name'] = [os.path.basename(x) for x in parsed['file_path']]

parsed['subject_id'] = [get_subject_id(x) for x in parsed['file_name']]
parsed['task'] = [set_task(x) for x in parsed['file_name']]
parsed['Need_Parsing'] = np.where(parsed['file_name'].isnull(), 1, 0)



In [24]:
parsed

,file_path,file_name,subject_id,task,Need_Parsing
0,./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table,A013_02_BBP_NORMAL_video.Table,A013,BBP_NORMAL,0
1,./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table,A013_02_NSM_BIGSMILE_video.Table,A013,NSM_BIGSMILE,0
2,./ALS/new/AF13/A013_02_NSM_OPEN_video.Table,A013_02_NSM_OPEN_video.Table,A013,NSM_OPEN,0
3,./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table,A013_02_NSM_SPREAD_video.Table,A013,NSM_SPREAD,0
4,./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table,A014_02_BBP_NORMAL_video.Table,A014,BBP_NORMAL,0
...,...,...,...,...,...
231,./Post-Stroke/old/S012/S012_02_NSM_OPEN_video....,S012_02_NSM_OPEN_video.Table,S012,NSM_OPEN,0
232,./Post-Stroke/old/S012/S012_02_NSM_SPREAD_vide...,S012_02_NSM_SPREAD_video.Table,S012,NSM_SPREAD,0
233,./Post-Stroke/old/S013/S013_02_BBP_NORMAL_audi...,S013_02_BBP_NORMAL_audio.Table,S013,BBP_NORMAL,0
234,./Post-Stroke/old/S013/S013_02_NSM_OPEN_video....,S013_02_NSM_OPEN_video.Table,S013,NSM_OPEN,0


In [25]:
test2 = test.merge(parsed, how= 'left', on=['subject_id', 'task'])
test2.loc[test2['Need_Parsing'].isnull(), 'Need_Parsing'] = 1
test2

,file_path_x,file_name_x,subject_id,task,Need_Clinical_Score,file_path_y,file_name_y,Need_Parsing
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,1,NaN,NaN,1.0
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,1,NaN,NaN,1.0
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,1,NaN,NaN,1.0
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,1,NaN,NaN,1.0
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,1,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
1240,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST,1,NaN,NaN,1.0
1241,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS,1,NaN,NaN,1.0
1242,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN,1,NaN,NaN,1.0
1243,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD,1,NaN,NaN,1.0


In [27]:
test2 = test2.drop(['file_path_y', 'file_name_y'],axis=1)



In [36]:
ids = ['A','K', 'OP', 'S', 'N', 'PD', 'P']

def pop_labeler(row):
    for i in ids:
        if i in row:
            if i == 'OP' or i == "S":         
                return 'Stroke'
            elif i == "A" or i == "P":
                return 'ALS'
            elif i == "PD":
                return 'PD'
            else:
                return 'HC'
            
test2['population'] = [pop_labeler(x) for x in test2['subject_id']]

In [37]:
test2.groupby(['population', 'Need_Clinical_Score']).count()

file_path_x  file_name_x  subject_id  task  \
population Need_Clinical_Score                                               
ALS        0                            119          119         119   119   
           1                            158          158         158   158   
HC         0                            145          145         145   145   
           1                            258          258         258   258   
PD         1                             91           91          91    91   
Stroke     0                            177          177         177   177   
           1                            289          289         289   289   

                                Need_Parsing  
population Need_Clinical_Score                
ALS        0                             119  
           1                             158  
HC         0                             145  
           1                             258  
PD         1                              91  
Stroke     0                             177  
           1                             289

In [38]:
test2.groupby(['population', 'Need_Parsing']).count()

file_path_x  file_name_x  subject_id  task  \
population Need_Parsing                                               
ALS        0.0                     6            6           6     6   
           1.0                   271          271         271   271   
HC         0.0                   140          140         140   140   
           1.0                   263          263         263   263   
PD         1.0                    91           91          91    91   
Stroke     0.0                   247          247         247   247   
           1.0                   219          219         219   219   

                         Need_Clinical_Score  
population Need_Parsing                       
ALS        0.0                             6  
           1.0                           271  
HC         0.0                           140  
           1.0                           263  
PD         1.0                            91  
Stroke     0.0                           247  
           1.0                           219

In [39]:
csv_name = 'clinical_score_dataset.csv'
test2.to_csv(csv_name)

## Testing random 

In [ ]:
print(data['file_name'][2])
data['file_name'][2].split('_')[2:4]

In [ ]:
#data = data[~data['subject_id'].isin(['001', '002', 'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04',
#       'PD05', 'PD06', 'PD07', 'PD08'])]

data['Need_Clinical_Score'] = data[~data['subject_id'].isin(['001', '002', 'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04',
       'PD05', 'PD06', 'PD07', 'PD08'])]


data['subject_id'].unique()